In [4]:
import os
import tarfile

In [ ]:
# Constants


In [ ]:
# Dataset Download: TVSum
!wget -O TVSum.tgz https://people.csail.mit.edu/yalesong/tvsum/tvsum50_ver_1_1.tgz

In [ ]:
# Extract the dataset
with tarfile.open("TVSum.tgz", 'r:gz') as tar_ref:
    tar_ref.extractall("./")

In [11]:
!ls ydata-tvsum50-v1_1

README			ydata-tvsum50-matlab.zip     ydata-tvsum50-video.zip
ydata-tvsum50-data.zip	ydata-tvsum50-thumbnail.zip


In [9]:
# Path to the extracted dataset (adjust if needed)
dataset_path = "ydata-tvsum50-v1_1"

# Example: List the video files in the dataset
video_files = [f for f in os.listdir(dataset_path) ]
# video_files = [f for f in os.listdir(dataset_path) if f.endswith(".mp4")]
print("Video files found:", video_files)

Video files found: ['README', 'ydata-tvsum50-video.zip', 'ydata-tvsum50-matlab.zip', 'ydata-tvsum50-thumbnail.zip', 'ydata-tvsum50-data.zip']
